## Fake News Detection

In [1]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Import the data
path = "../../Datasets/Fake_News/"
true_df = pd.read_csv(path + 'True.csv')
fake_df = pd.read_csv(path + 'Fake.csv')

In [3]:
#lable true news to 0 and fake news to 1
true_df['label'] = 0

In [4]:
fake_df['label'] = 1

In [5]:
# See first 5 rows of true news
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [6]:
# see first 5 rows of fake news
fake_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [7]:
# Extract important columns
true_df = true_df[['text','label']]
fake_df = fake_df[['text','label']]

In [8]:
# concat the true and fake news dataframe
dataset = pd.concat([true_df , fake_df])

In [9]:
dataset.shape

(44898, 2)

In [10]:
dataset.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0


### Null values

In [11]:
dataset.isnull().sum() # no null values

text     0
label    0
dtype: int64

### Balanced or Unbalanced dataset

In [12]:
dataset['label'].value_counts()

1    23481
0    21417
Name: label, dtype: int64

In [13]:
true_df.shape # true news

(21417, 2)

In [14]:
fake_df.shape # fake news

(23481, 2)

### Shuffle or Resample

In [15]:
dataset = dataset.sample(frac = 1)

In [16]:
dataset.head(20)

,text,label
12331,They had to know they had it coming Donald Tru...,1
3494,Trump mouthpiece Kayleigh McEnany attacked for...,1
6607,(Reuters) - Michigan Governor Rick Snyder said...,0
18685,COX S BAZAR (Reuters) - Grieving Rohingya Musl...,0
5761,Author Stephen King has been a longtime advoca...,1
15654,MANILA (Reuters) - A top Catholic archbishop o...,0
7355,"When you try and fail at everything you do, tr...",1
5969,After emerging victorious in the Democratic Pr...,1
9925,WASHINGTON (Reuters) - Republican presidential...,0
2812,(Reuters) - Co-host of MSNBC’s “Morning Joe” a...,0


In [17]:
import nltk

In [18]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [19]:
ps = WordNetLemmatizer()

In [20]:
stopwords = stopwords.words('english')

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dnyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
def cleaning_data(row):
    
    # convert text to into lower case
    row = row.lower() 
    
    # this line of code only take words from text and remove number and special character using RegX
    row = re.sub('[^a-zA-Z]' , ' ' , row)
    
    # split the data and make token.
    token = row.split() 
    
    # lemmatize the word and remove stop words like a, an , the , is ,are ...
    news = [ps.lemmatize(word) for word in token if not word in stopwords]  
    
    # finaly join all the token with space
    cleanned_news = ' '.join(news) 
    
    # return cleanned data
    return cleanned_news 

In [23]:
dataset['text'] = dataset['text'].apply(lambda x : cleaning_data(x))

In [24]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))

In [27]:
X = dataset.iloc[:35000,0]
y = dataset.iloc[:35000,1]

In [28]:
X.head()

12331    know coming donald trump scolded medium big sh...
3494     trump mouthpiece kayleigh mcenany attacked for...
6607     reuters michigan governor rick snyder said rea...
18685    cox bazar reuters grieving rohingya muslim ref...
5761     author stephen king longtime advocate gun cont...
Name: text, dtype: object

In [29]:
y.head()

12331    1
3494     1
6607     0
18685    0
5761     1
Name: label, dtype: int64

In [30]:
from sklearn.model_selection import train_test_split
train_X , test_X , train_y , test_y = train_test_split(X , y , test_size = 0.2 ,random_state = 0)

In [32]:
vec_train = vectorizer.fit_transform(train_X)

In [33]:
vec_train = vec_train.toarray()

In [34]:
vec_test = vectorizer.transform(test_X).toarray()

In [35]:
train_data = pd.DataFrame(vec_train , columns=vectorizer.get_feature_names())
test_data = pd.DataFrame(vec_test , columns= vectorizer.get_feature_names())

## Multinomial NB

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
from sklearn.metrics import accuracy_score,classification_report

In [38]:
clf = MultinomialNB()

In [39]:
clf.fit(train_data, train_y)
predictions  = clf.predict(test_data)

In [40]:
print(classification_report(test_y , predictions))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3340
           1       0.95      0.95      0.95      3660

    accuracy                           0.95      7000
   macro avg       0.95      0.95      0.95      7000
weighted avg       0.95      0.95      0.95      7000



Now predict on both train set

In [41]:
predictions_train = clf.predict(train_data)
print(classification_report(train_y , predictions_train))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13425
           1       0.96      0.96      0.96     14575

    accuracy                           0.96     28000
   macro avg       0.96      0.96      0.96     28000
weighted avg       0.96      0.96      0.96     28000



In [42]:
accuracy_score(train_y , predictions_train)

0.9600357142857143

In [43]:
accuracy_score(test_y , predictions)

0.9508571428571428

In [44]:
news = cleaning_data(str("Imposters posing as army personnel on the social media have been called out by the Indian Army as false news and disinformation."))

In [45]:
single_prediction = clf.predict(vectorizer.transform([news]).toarray())
single_prediction

array([1], dtype=int64)

# Save the Model 

In [46]:
import joblib 

In [47]:
joblib.dump(clf , 'model.pkl')

['model.pkl']

In [48]:
model = joblib.load('model.pkl')

In [49]:
model

MultinomialNB()